# Running Processing Jobs

A processing job serves as a method to enhance the utilization of ordered geospatial data by employing advanced processing algorithms. This process transforms the data into an analytics-ready state or extracts information for subsequent insight derivation.

- <a href="#setup">Set up the notebook</a>
- <a href="#single-input">Run Single Input Process</a>
- <a href="#job-list">View Jobs</a>
- <a href="#two-input">Run Two Input Process</a>

## <a id="setup"></a> Set up the notebook

### 1. Install prerequisites

In [ ]:
#!pip install up42-py
import up42, os
from up42 import processing_templates as templates
from up42 import processing as processes

### 2. Authenticate

Create a `credentials.json` file in a directory named `.up42` under your home directory by running the cell below. The path to the file will be `~/.up42/credentials.json`.

In [ ]:
# Define the directory path
up42_directory = os.path.expanduser("~/.up42")

# Create the directory if it doesn't exist
if not os.path.exists(up42_directory):
    os.makedirs(up42_directory)

# Specify the file path
credentials_file_path = os.path.join(up42_directory, "credentials.json")

# Check if the file already exists before creating it
if not os.path.exists(credentials_file_path):
    # Create an empty credentials.json file
    with open(credentials_file_path, "w") as credentials_file:
        print(f"The file {credentials_file_path} has been created.")
        pass
else:
    print(f"The file {credentials_file_path} already exists.")

1. Click the link above to the created file and paste the following code:
    ```json
    {
        "username": "<your-email-address>",
        "password": "<your-password>"
    }
    ```
2. Retrieve the email address and password used for logging into the console. Use them as values for username and password.
3. Save the `credentials.json` file.
4. Check that the authentication was successful as follows:

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

## <a id="single-input"></a> Run Single Input Process

Execute Jobs with one STAC input such as Pansharpening

### 1. Initialize a specific STAC Item

A better scenario would be to have the asset_id OR STAC item_id already in mind.
The input to the Template is a STAC Item object. Therefore, it should be from an asset delivery that is CNAM-compatible

In [ ]:
asset_id = ""
stac_item = up42.initialize_asset(asset_id).stac_items[0]

### 2. Prepare the parameters

In [ ]:
title = "Example_Pansharpen_SDK"
expected_cost = 100 #MODIFY

### 3. Create a Job Template

In [ ]:
template = templates.Pansharpening(
    title=title,
    item=stac_item
)
print(template)

### 4. Validate the Job

In [ ]:
if not template.is_valid:
    print(template.errors)

### 5. Get the Job Pricing

In [ ]:
print(template.cost)
assert template.cost.credits <= expected_cost

### 6. Execute & Track the Job

In [ ]:
job = template.execute()
job.track(wait=30, retries=6)

print(job.status)
print(job.collection_url)

## <a id="job-list"></a> View Jobs

Retrieve all jobs or a specific job available to user, with additional filtering capabilities

### 1. Get All Jobs

Set up parameters for filtering jobs

In [ ]:
workspace_id = "" #FILL IN
process_filter = [
    templates.Pansharpening.process_id,
    templates.DetectionChangeSpacept.process_id,
]
status_filter = [
    processes.JobStatus.CAPTURED, 
    processes.JobStatus.RELEASED
]
sort_filter = processes.JobSorting.credits.asc

Call the get jobs method, returning a generator object

In [ ]:
jobs = processes.Job.all(
#    process_id=process_filter
#    workspace_id=workspace_id,
#    sort_by=status_filter,
#    min_duration=min_duration,
#    max_duration=max_duration,
)
for i in jobs:  
    print(i)

### 2. Get a Specific Job

In [ ]:
queried_job = processes.Job.get(job.id)
print(queried_job)

## <a id="two-input"></a> Run Two Input Process

Execute Jobs with two STAC input such as Change Detection

In [ ]:
stac_item_id1= "" #FILL IN
stac_item_id2= "" #FILL IN

stac_item1 = pystac.get_item(stac_item_id1).to_dict()
stac_item2 = pystac.get_item(stac_item_id2).to_dict()

template = templates.HypervergePleiadesChangeDetection(
    title="SDK Detection Change Example", 
    items=[stac_item1, stac_item2]
)

And you would validate and execute the template in a similar manner to the first example